<a href="https://colab.research.google.com/github/c-labropoulos/NN-for-lithological-segmentation/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#!gdalinfo --version
!pip install geopandas
!pip install rasterio
!pip install pyshp
from osgeo import gdal
import pandas as pd
from osgeo import ogr
import rasterio
import geopandas as gpd
import shapefile as shp
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.plot import show
import numpy as np
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
shape_dir = "/content/drive/MyDrive/LithDatasSHP"
rst_list=list()
dest_name=list()
shpfiles = []
shapenames=[]
for dirpath, subdirs, files in os.walk(shape_dir):
    for x in files:
        if x.endswith(".shp"):
            shpfiles.append(os.path.join(dirpath, x))
            shapenames.append(x.split('.shp', 1)[0])


raster_dir='/content/drive/MyDrive/raster_to_be_used'
for dirpath, subdirs, files in os.walk(raster_dir):
    for x in files:
        rst_list.append(os.path.join(dirpath, x))
        dest_name.append(x.split('_rasterized_vector.tif', 1)[0])
rst_list.pop(0)  
dest_name.pop(0)    


shapesList =list()


for rst in dest_name:
 for shp in shpfiles:
   if rst in shp :
     shapesList.append(shp)

shapesList=list(dict.fromkeys(shapesList))
datadict = {}

for i in rst_list:
    datadict[i] = None

if len(shapesList)==len(datadict.keys()):
      print(len(shapesList))
else:
     print("not equal")

for  i in range(len(shapesList)):

   vector = gpd.read_file(shapesList[i])
   #  print("Vector : "+str(shapesList[i])+" opened ")
   datalist=vector['labels_Lit'].tolist()
   vector['labels_Lit'] = range(0,len(vector))
   vectdict={}
   for key in vector['labels_Lit']:
     for value in datalist:
        vectdict[key] = value
        datalist.remove(value)
        break
   
   for  key in   datadict:
      if list(datadict.keys()).index(key)== i:
        datadict[key]=vectdict
      




30


In [ ]:
#from pandas.tseries.frequencies import unique
def find_unique_values(d):
    unique_values = set()
    for value in d.values():
        if isinstance(value, dict):
            unique_values.update(find_unique_values(value))
        else:
            unique_values.add(value)
    unique_classes = {element: index for index, element in enumerate(unique_values)}
   # fin_unique_classes={v: k for k, v in unique_classes.items()}
    return unique_classes
unique=find_unique_values(datadict)
unique={v: k for k, v in unique.items()}
print(unique)


{0: nan, 1: 'Koites Potamon', 2: 'Argilikos Flisxis kai Keratolithoi', 3: 'Konoi Apotheseon', 4: 'Kollouvia Flisxi', 5: 'Psamitikos Flisxis', 6: 'Gnesioi', 7: 'Skliroi asvestolithoi', 8: 'Dolites', 9: 'Granitis', 10: 'Sxistolithoi', 11: 'Allouvia', 12: 'Kollouvia Gneusion', 13: 'Peridotites kai Gavroi', 14: 'Not available', 15: 'Miktos Flisxis', 16: 'Kollouvia Asvestolithon', 17: 'Tritogeneis apotheseis'}


In [ ]:

for key, value in unique.items():
    for sub_dict in datadict.values():
        if value in sub_dict.values():
            old_key = list(sub_dict.keys())[list(sub_dict.values()).index(value)]
            sub_dict[key] = sub_dict.pop(old_key)

for p_id, p_info in datadict.items():
    print("\n"+ str(p_id))
    
    for key in p_info:
        print(str(key) + ':', str(p_info[key]))
csvoutput= "/content/drive/MyDrive/data.csv"  
with open(csvoutput, 'w') as csvfile:
  writer = csv.writer(csvfile)
  for key, value in datadict.items():
      writer.writerow([key, value])


/content/drive/MyDrive/raster_to_be_used/Kirko_Bunaras_rasterized_vector.tif
6: Gnesioi
9: Granitis
17: Tritogeneis apotheseis

/content/drive/MyDrive/raster_to_be_used/Kithnos_A_rasterized_vector.tif
7: Skliroi asvestolithoi
10: Sxistolithoi
11: Allouvia

/content/drive/MyDrive/raster_to_be_used/Voras_rasterized_vector.tif
6: Gnesioi
7: Skliroi asvestolithoi

/content/drive/MyDrive/raster_to_be_used/Kithnos_B_rasterized_vector.tif
10: Sxistolithoi
11: Allouvia

/content/drive/MyDrive/raster_to_be_used/Monopigado_rasterized_vector.tif
7: Skliroi asvestolithoi
9: Granitis
10: Sxistolithoi
17: Tritogeneis apotheseis

/content/drive/MyDrive/raster_to_be_used/Kyprios_rasterized_vector.tif
10: Sxistolithoi
11: Allouvia
14: Not available
17: Tritogeneis apotheseis

/content/drive/MyDrive/raster_to_be_used/Kastro_rasterized_vector.tif
7: Skliroi asvestolithoi
11: Allouvia
17: Tritogeneis apotheseis

/content/drive/MyDrive/raster_to_be_used/Agios_Georgios_rasterized_vector.tif
7: Skliroi asve

In [ ]:

# Convert nested dictionary to DataFrame
df = pd.DataFrame.from_dict(datadict, orient="index")

# Print DataFrame
print(df)


                                                         6         9   \
/content/drive/MyDrive/raster_to_be_used/Kirko_...  Gnesioi  Granitis   
/content/drive/MyDrive/raster_to_be_used/Voras_...  Gnesioi       NaN   
/content/drive/MyDrive/raster_to_be_used/Petrot...  Gnesioi  Granitis   
/content/drive/MyDrive/raster_to_be_used/Kimmer...  Gnesioi  Granitis   
/content/drive/MyDrive/raster_to_be_used/LIMENA...  Gnesioi       NaN   
/content/drive/MyDrive/raster_to_be_used/Monopi...      NaN  Granitis   
/content/drive/MyDrive/raster_to_be_used/naxos_...      NaN  Granitis   
/content/drive/MyDrive/raster_to_be_used/Anafi_...      NaN  Granitis   
/content/drive/MyDrive/raster_to_be_used/Kyprio...      NaN       NaN   
/content/drive/MyDrive/raster_to_be_used/Kastro...      NaN       NaN   
/content/drive/MyDrive/raster_to_be_used/Agios_...      NaN       NaN   
/content/drive/MyDrive/raster_to_be_used/Amfilo...      NaN       NaN   
/content/drive/MyDrive/raster_to_be_used/Crete_... 

In [ ]:
# Print the size of the DataFrame
print('Number of rows: {}'.format(df.shape[0]))
print('Number of columns: {}'.format(df.shape[1]))

Number of rows: 30
Number of columns: 18


In [ ]:
# Convert the nested dictionary to a DataFrame
dft = pd.DataFrame.from_dict({k: {x:y for x,y in v.items()} for k,v in datadict.items()}, orient='index')

# Display the resulting DataFrame
print(dft.head)

<bound method NDFrame.head of                                                          6         9   \
/content/drive/MyDrive/raster_to_be_used/Kirko_...  Gnesioi  Granitis   
/content/drive/MyDrive/raster_to_be_used/Voras_...  Gnesioi       NaN   
/content/drive/MyDrive/raster_to_be_used/Petrot...  Gnesioi  Granitis   
/content/drive/MyDrive/raster_to_be_used/Kimmer...  Gnesioi  Granitis   
/content/drive/MyDrive/raster_to_be_used/LIMENA...  Gnesioi       NaN   
/content/drive/MyDrive/raster_to_be_used/Monopi...      NaN  Granitis   
/content/drive/MyDrive/raster_to_be_used/naxos_...      NaN  Granitis   
/content/drive/MyDrive/raster_to_be_used/Anafi_...      NaN  Granitis   
/content/drive/MyDrive/raster_to_be_used/Kyprio...      NaN       NaN   
/content/drive/MyDrive/raster_to_be_used/Kastro...      NaN       NaN   
/content/drive/MyDrive/raster_to_be_used/Agios_...      NaN       NaN   
/content/drive/MyDrive/raster_to_be_used/Amfilo...      NaN       NaN   
/content/drive/MyDriv

In [ ]:
# Print the size of the DataFrame
print('Number of rows: {}'.format(dft.shape[0]))
print('Number of columns: {}'.format(dft.shape[1]))

Number of rows: 30
Number of columns: 18


In [ ]:
df.to_csv('mydata.csv', index=False)

In [ ]:
import pandas as pd

#nestdict = {'location1': {'key1': 'lithology1', 'key2': 'lithology2', 'key3': 'lithology3', 'key4': 'lithology4'},
         #   'location2': {'key1': 'lithology1', 'key2': 'lithology2', 'key3': 'lithology3', 'key4': 'lithology4'}}

df = pd.DataFrame.from_dict(datadict, orient='index')
df = df.rename_axis('location').reset_index()

df = pd.melt(df, id_vars=['location'], var_name='lithology_key', value_name='lithology')
df = df.sort_values(by=['location', 'lithology_key']).reset_index(drop=True)
print('Number of rows: {}'.format(dft.shape[0]))
print('Number of columns: {}'.format(dft.shape[1]))


Number of rows: 30
Number of columns: 18


In [ ]:
import os
from osgeo import gdal
import glob

# Set the path to the folder containing the raster files
folder_path = "/content/drive/MyDrive/raster_to_be_used"

# Find all the raster files in the folder
raster_files = glob.glob(os.path.join(folder_path, "*.tif"))

# Loop through each raster file and get the number of bands
for raster_file in raster_files:
    raster = gdal.Open(raster_file)
    num_bands = raster.RasterCount
    print(f"Number of bands in {raster_file}: {num_bands}")


Number of bands in /content/drive/MyDrive/raster_to_be_used/Desfina_A_rasterized_vector.tif: 1
Number of bands in /content/drive/MyDrive/raster_to_be_used/Kirko_Bunaras_rasterized_vector.tif: 1
Number of bands in /content/drive/MyDrive/raster_to_be_used/Kithnos_A_rasterized_vector.tif: 1
Number of bands in /content/drive/MyDrive/raster_to_be_used/Voras_rasterized_vector.tif: 1
Number of bands in /content/drive/MyDrive/raster_to_be_used/Kithnos_B_rasterized_vector.tif: 1
Number of bands in /content/drive/MyDrive/raster_to_be_used/Monopigado_rasterized_vector.tif: 1
Number of bands in /content/drive/MyDrive/raster_to_be_used/Kyprios_rasterized_vector.tif: 1
Number of bands in /content/drive/MyDrive/raster_to_be_used/Kastro_rasterized_vector.tif: 1
Number of bands in /content/drive/MyDrive/raster_to_be_used/Agios_Georgios_rasterized_vector.tif: 1
Number of bands in /content/drive/MyDrive/raster_to_be_used/Amfilochia_rasterized_vector.tif: 1
Number of bands in /content/drive/MyDrive/raster